https://www.kaggle.com/code/manovirat/timeseries-using-prophet-hyperparameter-tuning

In [22]:
import pandas as pd
from prophet import Prophet
from prophet.diagnostics import cross_validation
from sklearn.model_selection import ParameterGrid
from prophet.diagnostics import performance_metrics
import random
import numpy as np
from sklearn.model_selection import TimeSeriesSplit

In [23]:
params_grid = {'seasonality_mode':('multiplicative','additive'),
               # set changepoint_prior_scale to 0.5 with steps of 0.005
               'changepoint_prior_scale':[x for x in np.arange(0.001, 0.5, 0.1)],
               'seasonality_prior_scale':[x for x in np.arange(0.01, 10, 2.5)],
              'holidays_prior_scale':[x for x in np.arange(0.01, 10, 2.5)]}

grid = ParameterGrid(params_grid)
cnt = 0
for p in grid:
    cnt = cnt+1

print('Total Possible Models',cnt)

Total Possible Models 160


In [24]:
#Define MASE Metric
def mean_absolute_scaled_error(y_true, y_pred):
    mase=0
    # Define numerator as the forecast error
    numerator = (np.abs(y_true - y_pred))

    # Define denominator as the mean absolute error of the in-sample one-step naive forecast
    y_true_ohne_1 = y_true[1:].reset_index(drop=True)
    y_true_ohne_ende = y_true[:-1].reset_index(drop=True)
    denominator = np.mean(np.abs(y_true_ohne_1 - y_true_ohne_ende))

    mase = np.mean(np.abs(numerator / denominator))

    return mase

In [25]:
# Load the data
data = pd.read_parquet('/Users/paddy/Documents/GitHub/Masterthesis_ML/data/03_032022_032023_taxi_rides.parquet')

# Convert the date column to datetime
data['date'] = pd.to_datetime(data['date'],format='%d/%m/%y %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:%S') 


In [26]:
# Feature Engineering

# Add a column with the time
data['time'] = [x for x in range(0, len(data))]

# Rename column count to y
data = data.rename(columns={'count': 'y'})
data = data.rename(columns={'date': 'ds'})


train_data = data

In [27]:
cv_folds = TimeSeriesSplit(n_splits=5)

#convert data to only date not time
cv_data = train_data.copy()

#convert to timestamp with date format
#cv_data.ds = pd.to_datetime(cv_data.ds).dt.date

print(cv_data.ds)   

#Define cutoff dates for cross validation at end of train_data/cv_folds
cutoffs = []
# for train_index, test_index in cv_folds.split(train_data):
#     cutoffs.append(train_data.ds.iloc[test_index].max())
#     #convert to timestamp
#     cutoffs = pd.to_datetime(cutoffs)
    
# print(cutoffs)

for i, (train_index, test_index) in enumerate(cv_folds.split(cv_data)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")
    #get the date for the last train index
    cutoffs.append(cv_data.ds.iloc[train_index].max())
    #only date not time
    #cutoffs = pd.to_datetime(cutoffs).date

print(cutoffs)

# Define cutoffs for cross validation with 5 folds every two months with first cut on 2022-06-01, last day 2023-01-15 and horizon of 15 days
#cutoffs = pd.to_datetime(['2022-06-01', '2022-08-01', '2022-10-01', '2022-12-01', '2023-01-15'])
# print(cutoffs)


0       2022-04-01 00:00:00
1       2022-04-01 01:00:00
2       2022-04-01 02:00:00
3       2022-04-01 03:00:00
4       2022-04-01 04:00:00
               ...         
8755    2023-03-31 19:00:00
8756    2023-03-31 20:00:00
8757    2023-03-31 21:00:00
8758    2023-03-31 22:00:00
8759    2023-03-31 23:00:00
Name: ds, Length: 8760, dtype: object
Fold 0:
  Train: index=[   0    1    2 ... 1457 1458 1459]
  Test:  index=[1460 1461 1462 ... 2917 2918 2919]
Fold 1:
  Train: index=[   0    1    2 ... 2917 2918 2919]
  Test:  index=[2920 2921 2922 ... 4377 4378 4379]
Fold 2:
  Train: index=[   0    1    2 ... 4377 4378 4379]
  Test:  index=[4380 4381 4382 ... 5837 5838 5839]
Fold 3:
  Train: index=[   0    1    2 ... 5837 5838 5839]
  Test:  index=[5840 5841 5842 ... 7297 7298 7299]
Fold 4:
  Train: index=[   0    1    2 ... 7297 7298 7299]
  Test:  index=[7300 7301 7302 ... 8757 8758 8759]
['2022-05-31 19:00:00', '2022-07-31 15:00:00', '2022-09-30 11:00:00', '2022-11-30 07:00:00', '2023-01-30

In [28]:
MASE_list = []
best_mase = 100000000
best_params = None
for p in grid:
    test = pd.DataFrame()
    print(p)
    random.seed(0)
    train_model =Prophet(changepoint_prior_scale = p['changepoint_prior_scale'],
                        holidays_prior_scale = p['holidays_prior_scale'],
                        seasonality_prior_scale = p['seasonality_prior_scale'],
                        seasonality_mode = p['seasonality_mode'],
                        weekly_seasonality=True,
                        daily_seasonality = True,
                        yearly_seasonality = False,
                        )
    
    train_model.add_regressor('time', standardize=False)


    train_model.fit(train_data)
    
    df_cv = cross_validation(train_model, period='60 days', horizon='30 days', parallel="processes")


    # X and y to pandas dataframe
    X_train = pd.DataFrame(df_cv)

    y_true = df_cv['y'].values
    #make y_true as dataframe
    y_true = pd.DataFrame(y_true)

    y_pred = df_cv['yhat'].values
    #make y_pred as dataframe
    y_pred = pd.DataFrame(y_pred)

    MASE_list.append(mean_absolute_scaled_error(y_true, y_pred))
    
    # print('RMSE:',np.sqrt(MSE))
    # RMSE_list.append(np.sqrt(MSE))

# Find the best parameters
tuning_results = pd.DataFrame(grid)
tuning_results['mase'] = MASE_list
print(tuning_results)

{'changepoint_prior_scale': 0.001, 'holidays_prior_scale': 0.01, 'seasonality_mode': 'multiplicative', 'seasonality_prior_scale': 0.01}


11:34:51 - cmdstanpy - INFO - Chain [1] start processing
11:34:56 - cmdstanpy - INFO - Chain [1] done processing
11:34:57 - cmdstanpy - INFO - Chain [1] start processing
11:34:57 - cmdstanpy - INFO - Chain [1] start processing
11:34:57 - cmdstanpy - INFO - Chain [1] start processing
11:34:58 - cmdstanpy - INFO - Chain [1] start processing
11:34:58 - cmdstanpy - INFO - Chain [1] start processing
11:34:58 - cmdstanpy - INFO - Chain [1] done processing
11:34:58 - cmdstanpy - INFO - Chain [1] done processing
11:34:58 - cmdstanpy - INFO - Chain [1] done processing
11:34:59 - cmdstanpy - INFO - Chain [1] done processing
11:35:01 - cmdstanpy - INFO - Chain [1] done processing


                    ds       yhat  yhat_lower  yhat_upper   y  \
0  2022-07-05 00:00:00  15.845919   -3.396010   35.081609   6   
1  2022-07-05 01:00:00  15.098855   -4.285777   33.365393   6   
2  2022-07-05 02:00:00  14.567471   -3.961365   32.272153   0   
3  2022-07-05 03:00:00  14.357241   -4.545017   33.342303   0   
4  2022-07-05 04:00:00  14.348024   -5.252702   34.387854   0   
5  2022-07-05 05:00:00  14.404833   -3.963558   33.861377   1   
6  2022-07-05 06:00:00  14.559780   -3.295534   33.498280   1   
7  2022-07-05 07:00:00  14.969042   -4.281671   35.268524  10   
8  2022-07-05 08:00:00  15.695307   -2.861653   34.366499  12   
9  2022-07-05 09:00:00  16.556845   -4.125794   34.400817  33   
10 2022-07-05 10:00:00  17.221997   -2.013475   36.221913  54   
11 2022-07-05 11:00:00  17.475008   -2.878824   35.188795  17   
12 2022-07-05 12:00:00  17.394475   -2.718240   36.665464  46   
13 2022-07-05 13:00:00  17.261750   -2.294118   34.505549  37   
14 2022-07-05 14:00:00  1

11:35:02 - cmdstanpy - INFO - Chain [1] start processing
11:35:03 - cmdstanpy - INFO - Chain [1] done processing
11:35:04 - cmdstanpy - INFO - Chain [1] start processing
11:35:04 - cmdstanpy - INFO - Chain [1] start processing
11:35:04 - cmdstanpy - INFO - Chain [1] start processing
11:35:04 - cmdstanpy - INFO - Chain [1] start processing
11:35:05 - cmdstanpy - INFO - Chain [1] start processing
11:35:05 - cmdstanpy - INFO - Chain [1] done processing
11:35:05 - cmdstanpy - INFO - Chain [1] done processing
11:35:05 - cmdstanpy - INFO - Chain [1] done processing
11:35:06 - cmdstanpy - INFO - Chain [1] done processing
11:35:08 - cmdstanpy - INFO - Chain [1] done processing


                    ds       yhat  yhat_lower  yhat_upper   y  \
0  2022-07-05 00:00:00  13.943409    1.903597   26.165191   6   
1  2022-07-05 01:00:00   4.311192   -7.665208   15.791610   6   
2  2022-07-05 02:00:00  -2.481149  -13.914506    9.522148   0   
3  2022-07-05 03:00:00  -5.197411  -16.242480    6.053790   0   
4  2022-07-05 04:00:00  -5.361470  -17.943748    6.901019   0   
5  2022-07-05 05:00:00  -4.601517  -16.070916    7.533857   1   
6  2022-07-05 06:00:00  -2.432860  -14.805446    9.543117   1   
7  2022-07-05 07:00:00   3.128213   -9.512108   14.625776  10   
8  2022-07-05 08:00:00  12.812600    0.686273   24.731322  12   
9  2022-07-05 09:00:00  24.236314   11.721956   35.330325  33   
10 2022-07-05 10:00:00  33.129812   20.485406   44.756019  54   
11 2022-07-05 11:00:00  36.695848   24.965936   48.858742  17   
12 2022-07-05 12:00:00  35.856182   23.938378   48.662898  46   
13 2022-07-05 13:00:00  34.121412   22.270594   46.343686  37   
14 2022-07-05 14:00:00  3

...

                    ds       yhat  yhat_lower  yhat_upper   y  \
0  2022-07-05 00:00:00  15.910062    4.821532   26.787590   6   
1  2022-07-05 01:00:00   8.290073   -2.154330   19.177842   6   
2  2022-07-05 02:00:00   2.889646   -7.993956   13.748937   0   
3  2022-07-05 03:00:00   0.723801  -10.330637   11.860837   0   
4  2022-07-05 04:00:00   0.586654  -10.532799   11.233515   0   
5  2022-07-05 05:00:00   1.166574  -10.387600   11.787954   1   
6  2022-07-05 06:00:00   2.832350   -8.461149   13.768651   1   
7  2022-07-05 07:00:00   7.161666   -4.471918   18.357986  10   
8  2022-07-05 08:00:00  14.749647    3.741327   25.201166  12   
9  2022-07-05 09:00:00  23.714928   13.294140   34.478878  33   
10 2022-07-05 10:00:00  30.672167   19.776791   41.630224  54   
11 2022-07-05 11:00:00  33.409792   23.020775   43.790510  17   
12 2022-07-05 12:00:00  32.682782   21.313597   44.258850  46   
13 2022-07-05 13:00:00  31.303542   19.582072   41.773526  37   
14 2022-07-05 14:00:00  3

11:50:08 - cmdstanpy - INFO - Chain [1] start processing
11:50:08 - cmdstanpy - INFO - Chain [1] done processing
11:50:09 - cmdstanpy - INFO - Chain [1] start processing
11:50:09 - cmdstanpy - INFO - Chain [1] start processing
11:50:09 - cmdstanpy - INFO - Chain [1] done processing
11:50:09 - cmdstanpy - INFO - Chain [1] done processing
11:50:09 - cmdstanpy - INFO - Chain [1] start processing
11:50:09 - cmdstanpy - INFO - Chain [1] start processing
11:50:10 - cmdstanpy - INFO - Chain [1] done processing
11:50:10 - cmdstanpy - INFO - Chain [1] done processing
11:50:10 - cmdstanpy - INFO - Chain [1] start processing
11:50:10 - cmdstanpy - INFO - Chain [1] done processing


                    ds       yhat  yhat_lower  yhat_upper   y  \
0  2022-07-05 00:00:00  15.899428    5.166898   26.682158   6   
1  2022-07-05 01:00:00   8.381278   -2.546608   19.023102   6   
2  2022-07-05 02:00:00   3.069926   -7.558932   14.106671   0   
3  2022-07-05 03:00:00   0.966196   -9.923869   11.915720   0   
4  2022-07-05 04:00:00   0.870717  -10.234735   11.435205   0   
5  2022-07-05 05:00:00   1.480542   -8.616689   11.361714   1   
6  2022-07-05 06:00:00   3.158795   -7.424945   14.470094   1   
7  2022-07-05 07:00:00   7.467867   -2.783837   18.445933  10   
8  2022-07-05 08:00:00  14.999243    4.130432   24.726103  12   
9  2022-07-05 09:00:00  23.893736   13.155552   35.246387  33   
10 2022-07-05 10:00:00  30.803229   20.761273   42.111241  54   
11 2022-07-05 11:00:00  33.540270   22.781963   44.595089  17   
12 2022-07-05 12:00:00  32.851832   21.952347   43.712861  46   
13 2022-07-05 13:00:00  31.519467   20.506074   42.486905  37   
14 2022-07-05 14:00:00  3

11:50:11 - cmdstanpy - INFO - Chain [1] start processing
11:50:11 - cmdstanpy - INFO - Chain [1] done processing
11:50:12 - cmdstanpy - INFO - Chain [1] start processing
11:50:12 - cmdstanpy - INFO - Chain [1] start processing
11:50:12 - cmdstanpy - INFO - Chain [1] start processing
11:50:12 - cmdstanpy - INFO - Chain [1] done processing
11:50:12 - cmdstanpy - INFO - Chain [1] done processing
11:50:12 - cmdstanpy - INFO - Chain [1] done processing
11:50:12 - cmdstanpy - INFO - Chain [1] start processing
11:50:13 - cmdstanpy - INFO - Chain [1] done processing
11:50:13 - cmdstanpy - INFO - Chain [1] start processing
11:50:13 - cmdstanpy - INFO - Chain [1] done processing


                    ds       yhat  yhat_lower  yhat_upper   y  \
0  2022-07-05 00:00:00  15.898966    5.421866   26.747692   6   
1  2022-07-05 01:00:00   8.380459   -2.160902   18.361956   6   
2  2022-07-05 02:00:00   3.068856   -7.075839   13.335653   0   
3  2022-07-05 03:00:00   0.965025   -9.233203   10.714593   0   
4  2022-07-05 04:00:00   0.869539   -9.536237   11.856526   0   
5  2022-07-05 05:00:00   1.479392   -9.509025   12.263028   1   
6  2022-07-05 06:00:00   3.157724   -8.593876   13.733945   1   
7  2022-07-05 07:00:00   7.466997   -3.488794   18.886599  10   
8  2022-07-05 08:00:00  14.998721    3.891670   25.945451  12   
9  2022-07-05 09:00:00  23.893623   12.515201   34.751775  33   
10 2022-07-05 10:00:00  30.803433   19.663440   41.601614  54   
11 2022-07-05 11:00:00  33.540598   23.184838   44.612175  17   
12 2022-07-05 12:00:00  32.852126   22.023029   43.973757  46   
13 2022-07-05 13:00:00  31.519694   20.604412   41.912955  37   
14 2022-07-05 14:00:00  3

In [29]:
best_params = grid[np.argmin(MASE_list)]
print('Best Parameters',best_params)
print('Best MASE',np.min(MASE_list))
print(MASE_list)

Best Parameters {'seasonality_prior_scale': 7.51, 'seasonality_mode': 'additive', 'holidays_prior_scale': 2.51, 'changepoint_prior_scale': 0.201}
Best MASE 0.761653916740978
[0.9958491310436506, 0.836436620005614, 0.8384868889789912, 0.8048542654347517, 0.9146240937375358, 0.9287696420018008, 0.9281209390802337, 0.9131771239527938, 0.9969514078129632, 0.8310275393740464, 0.8229673668363482, 0.8477404415717127, 0.918774757423814, 0.9133206289927448, 0.9642105701167991, 0.9123411492223473, 0.978363476198712, 0.8166742326659526, 0.8223209420341459, 0.841303207326941, 0.9383877581161502, 0.9159052766017827, 0.9140487545242114, 0.9131622258064065, 0.9768955462144219, 0.8317194999518996, 0.8253919383647415, 0.8460757882754202, 0.9166668359539596, 0.9138106191344467, 0.9167426785263432, 0.9118925597608534, 1.3171884569194625, 0.8539262339469376, 0.77119592721029, 0.7831691118298141, 0.7795506275944162, 0.776200984168527, 0.7743027027301415, 0.7727190613946161, 1.2812623688927722, 0.7743607553